In [1]:
import numpy as np
np.random.seed(10)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
n_estimator = 10

In [3]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
y=train.target.values
X=train.drop(["target"],axis=1)

In [5]:
#切割训练集，一部分数据作为验证集，一部分数据作为训练集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
#将上一步得到的训练集切割，一半用来训练GBDT模型，一般用来训练LR模型
X_train_gbdt, X_train_lr, y_train_gbdt, y_train_lr = train_test_split(X_train, y_train, test_size=0.5)

In [6]:
# 基于GBDT监督变换
grd = GradientBoostingClassifier(n_estimators=n_estimator)
grd.fit(X_train, y_train)
# 得到OneHot编码
grd_enc = OneHotEncoder(categories='auto')

temp = grd.apply(X_train)
np.set_printoptions(threshold=np.inf)  
grd_enc.fit(grd.apply(X_train)[:, :, 0])
#print(grd_enc.get_feature_names()) # 查看每一列对应的特征
# 使用OneHot编码作为特征，训练LR
grd_lm = LogisticRegression(solver='lbfgs', max_iter=1000)
grd_lm.fit(grd_enc.transform(grd.apply(X_train_lr)[:, :, 0]), y_train_lr)
# 使用LR进行预测
y_pred_grd_lm = grd_lm.predict_proba(grd_enc.transform(grd.apply(X_test)[:, :, 0]))[:, 1]

In [15]:
logloss = log_loss(y_test,y_pred_grd_lm)
logloss

0.15472972193518425

In [20]:
# 预测test
y_pred = grd_lm.predict_proba(grd_enc.transform(grd.apply(test)[:, :, 0]))[:, 1]

In [52]:
ids=x.values
result=pd.DataFrame(data=zip(ids,y_pred),columns=["id","target"])

In [55]:
result.head(10)

,id,target
0,0,0.028554
1,1,0.024959
2,2,0.027464
3,3,0.017050
4,4,0.032396
5,5,0.050432
6,6,0.038477
7,8,0.029886
8,10,0.070164
9,11,0.053077


In [57]:
result.to_csv("action1.csv")